# Lab 5

## Context Engineering

We want to develop several types of Memory:

1. Long Term Memory - graph

A knowledge graph as a persistent store of entities

2. Long Term Memory - knowledge

A RAG database of Q&A and any other useful information

3. Permanent context

Summary and linkedin profile included in everything

4. FAQ

A list of questions and answers

In [ ]:
from dotenv import load_dotenv
from agents.mcp import MCPServerStdio
import sqlite3
load_dotenv(override=True)

## Websites where you can find MCP Servers


- https://mcp.so
- https://glama.ai
- https://smithery.ai

## First, look at this Knowledge Graph MCP server built on libsql

https://glama.ai/mcp/servers/@joleyline/mcp-memory-libsql

In [ ]:
memory_graph_params = {"command": "npx","args": ["-y", "mcp-memory-libsql"],"env": {"LIBSQL_URL": "file:./twin/memory/graph.db"}}

In [ ]:
async with MCPServerStdio(params=memory_graph_params, client_session_timeout_seconds=30) as memory_graph:
    memory_graph_tools = await memory_graph.session.list_tools()

memory_graph_tools.tools

## Next, here is a Vector Store RAG memory built on Qdrant

https://glama.ai/mcp/servers/@qdrant/mcp-server-qdrant

In [ ]:
memory_rag_params = {
    "command": "uvx",
    "args": ["mcp-server-qdrant"],
    "env": {
        "QDRANT_LOCAL_PATH": "./twin/memory/knowledge/",
        "COLLECTION_NAME": "knowledge",
        "EMBEDDING_MODEL": "sentence-transformers/all-MiniLM-L6-v2"
    }
}

In [ ]:
async with MCPServerStdio(params=memory_rag_params, client_session_timeout_seconds=30) as memory_rag:
    memory_rag_tools = await memory_rag.session.list_tools()

memory_rag_tools.tools

## And we'll also make some tools

In [ ]:
DB = "./twin/memory/questions.db"

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS questions (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            question TEXT,
            answer TEXT
        )
    ''')
    conn.commit()

In [ ]:
def record_question_with_no_answer(question: str) -> str:
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute("INSERT INTO questions (question, answer) VALUES (?, NULL)", (question,))
        conn.commit()
        return "Recorded question with no answer"

def get_questions_with_no_answer() -> str:
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT id, question FROM questions WHERE answer IS NULL")
        rows = cursor.fetchall()
        if rows:
            return "\n".join(f"Question id {row[0]}: {row[1]}" for row in rows)
        else:
            return "No questions with no answer found"

def get_questions_with_answer() -> str:
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT question, answer FROM questions WHERE answer IS NOT NULL")
        rows = cursor.fetchall()
        return "\n".join(f"Question: {row[0]}\nAnswer: {row[1]}\n" for row in rows)

def record_answer_to_question(id: int, answer: str) -> str:
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute("UPDATE questions SET answer = ? WHERE id = ?", (answer, id))
        conn.commit()
        return "Recorded answer to question"

In [ ]:
get_questions_with_no_answer()

In [ ]:
print(get_questions_with_answer())

In [ ]:
record_question_with_no_answer("What is your favorite musical instrument?")

In [ ]:
record_answer_to_question(3, "Banana")